This notebook contains the code neccessary for cleaning up the player-level data, both the data set scraped for players per game stats as well as the data set scraped for players' per 100 possessions stats, players.csv and per_100.csv, respectively. Note: some cleaning concerning datatypes already took place in the webscraping notebook as well. 

## Handling NaNs

In [8]:
import numpy as np
import pandas as pd

In [9]:
pergame = pd.read_csv("data/pergame.csv")
pergame = pergame.drop(columns = 'Unnamed: 0')

pergame.head(3)

,Last Name,First Name,Full Name,Year,Pos,Age,Tm,G,GS,MPG,...,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS
0,Abrines,Álex,Álex Abrines,2018.0,SG,25.0,OKC,31.0,2.0,19.0,...,0.923,0.2,1.4,1.5,0.6,0.5,0.2,0.5,1.7,5.3
1,Acy,Quincy,Quincy Acy,2018.0,PF,28.0,PHO,10.0,0.0,12.3,...,0.700,0.3,2.2,2.5,0.8,0.1,0.4,0.4,2.4,1.7
2,Adams,Jaylen,Jaylen Adams,2018.0,PG,22.0,ATL,34.0,1.0,12.6,...,0.778,0.3,1.4,1.8,1.9,0.4,0.1,0.8,1.3,3.2


In [10]:
print(len(pergame.columns))
pergame.columns

32


Index(['Last Name', 'First Name', 'Full Name', 'Year', 'Pos', 'Age', 'Tm', 'G',
       'GS', 'MPG', 'FG', 'FGA', 'FG%', '3P', '3PA', '3P%', '2P', '2PA', '2P%',
       'eFG%', 'FT', 'FTA', 'FT%', 'ORB', 'DRB', 'TRB', 'AST', 'STL', 'BLK',
       'TOV', 'PF', 'PTS'],
      dtype='object')

As the column list above should indicate, we have a lot of possible features to consider when grouping players. To begin whittling down our options, it's an important first step to see where we might have null values, as many algorithms won't take too kindly to nulls.

In [11]:
nanlist = pergame.isnull().sum()
nanlist = nanlist.where(nanlist > 0).dropna()
print('Number of features containing NaN values:',(len(nanlist)),'\n')
print(nanlist)

Number of features containing NaN values: 6 

GS       694.0
FG%       98.0
3P%     3623.0
2P%      150.0
eFG%      98.0
FT%      846.0
dtype: float64


Now let's look more closely at the features: GS, Games Started, is an important metric for a player's use; he is valued as a starter compared to a bench player. Upon further inspection, there are two years (1979 and 1980, the earlist seasons we have) where GS data is simply not entered for most players, so we'll need to approximate it ourselves based on what's logical. Because we DO have the Games Played data, we can take the few years around our concerning years and establish Games Started as a ratio of Games Played. The code for establishing the ratio and filling in our NaNs follows below:

In [12]:
players1980 = pergame[pergame.Year == 1980]
players1979 = pergame[pergame.Year == 1979]
players1980.GS.isnull().sum(), players1979.GS.isnull().sum()

(350, 344)

In [13]:
next3 = pergame[(pergame.Year >= 1981) & (pergame.Year < 1984)]
Gmean = np.mean(next3.G)
GSmean = np.mean(next3.GS)
ratio = GSmean/Gmean
print('ratio of Games Started to Games Played:',round(ratio,2))

ratio of Games Started to Games Played: 0.47


In [14]:
pergame['GS'] = pergame.apply(
    lambda row: round(row['G'] * ratio,1) if np.isnan(row['GS']) and row['G'] < 60 else row['GS'],
    axis=1
)
pergame['GS'] = pergame['GS'].fillna(pergame['G'] - 5)

In [15]:
pergame[pergame.Year == 1980]

,Last Name,First Name,Full Name,Year,Pos,Age,Tm,G,GS,MPG,...,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS
19579,Abdul-Jabbar*,Kareem,Kareem Abdul-Jabbar*,1980.0,C,33.0,LAL,80.0,75.0,37.2,...,0.766,2.5,7.8,10.3,3.4,0.7,2.9,3.1,3.1,26.2
19580,Abernethy,Tom,Tom Abernethy,1980.0,SF,26.0,TOT,39.0,18.5,7.6,...,0.591,0.5,0.7,1.2,0.5,0.2,0.1,0.2,0.9,1.6
19581,Abernethy,Tom,Tom Abernethy,1980.0,SF,26.0,GSW,10.0,4.7,3.9,...,0.667,0.1,0.7,0.8,0.1,0.1,0.0,0.2,0.5,0.4
19582,Abernethy,Tom,Tom Abernethy,1980.0,SF,26.0,IND,29.0,13.7,8.9,...,0.579,0.7,0.7,1.4,0.6,0.2,0.1,0.2,1.0,2.0
19583,Adams,Alvan,Alvan Adams,1980.0,C,26.0,PHO,75.0,70.0,27.4,...,0.768,2.1,5.2,7.3,4.6,1.4,0.9,3.0,3.0,14.9
19584,Allums,Darrell,Darrell Allums,1980.0,PF,22.0,DAL,22.0,10.4,12.5,...,0.591,0.9,2.1,3.0,1.1,0.2,0.4,1.0,2.3,2.7
19585,Archibald*,Tiny,Tiny Archibald*,1980.0,PG,32.0,BOS,80.0,72.0,35.3,...,0.816,0.5,1.8,2.2,7.7,0.9,0.2,3.3,2.5,13.8
19586,Awtrey,Dennis,Dennis Awtrey,1980.0,C,32.0,SEA,47.0,22.3,12.9,...,0.700,0.7,1.6,2.3,1.1,0.3,0.2,0.7,1.8,2.2
19587,B. Free,World,World B. Free,1980.0,SG,27.0,GSW,65.0,60.0,36.5,...,0.814,0.7,1.7,2.4,5.6,1.3,0.2,3.0,2.8,24.1
19588,Bailey,James,James Bailey,1980.0,PF,23.0,SEA,82.0,77.0,31.0,...,0.709,2.3,5.1,7.4,1.2,0.9,1.7,2.7,4.0,14.0


While this is a crude fix, we've at least respected that players are either starting all season or their starts are a function of their games played.

The remaining NaNs are in columns that give metrics of various rates or percentages such as FT% or 3P%. If a player didn't put up a single free throw (and therefore didn't have a chance to make or miss a free throw), then of course his FT% isn't a computable number, but it certainly isn't 0 and shouldn't be calculated as such. When we later control our data for only players of interest (players who play most games in any particular season), these NaNs will be accounted for. In other words, if we're only looking at players who have played 40+ games, we probably won't see too many NaNs in the 2P%, for instance, as it'll be very likely that such players will have taken at least one 2P shot.

Our final NaNs are below:

In [16]:
nanlist = pergame.isnull().sum()
nanlist = nanlist.where(nanlist > 0).dropna()
print('Number of features containing NaN values:',(len(nanlist)),'\n')
print(nanlist)

Number of features containing NaN values: 5 

FG%       98.0
3P%     3623.0
2P%      150.0
eFG%      98.0
FT%      846.0
dtype: float64


Now we repeat the process with our 'per_100.csv' file, which contain players stats but this time standardized over 100 possessions to account for usage rate.

In [17]:
per_100 = pd.read_csv("data/per_100.csv")
per_100 = per_100.drop(columns = ['Unnamed: 0','Unnamed: 32'])


In [18]:
nanlist = per_100.isnull().sum()
nanlist = nanlist.where(nanlist > 0).dropna()
print('Number of features containing NaN values:',(len(nanlist)),'\n')
print(nanlist)

Number of features containing NaN values: 37 

GS        694.0
FG          5.0
FGA         5.0
FG%        98.0
3P          5.0
3PA         5.0
3P%      3623.0
2P          5.0
2PA         5.0
2P%       150.0
FT          5.0
FTA         5.0
FT%       846.0
ORB         5.0
DRB         5.0
TRB         5.0
AST         5.0
STL         5.0
BLK         5.0
TOV         5.0
PF          5.0
PTS         5.0
ORtg       55.0
DRtg        5.0
PER         5.0
TS%        86.0
3PAr       98.0
FTr        98.0
ORB%        5.0
DRB%        5.0
TRB%        5.0
AST%        5.0
STL%        5.0
BLK%        5.0
TOV%       71.0
USG%        5.0
WS/48       5.0
dtype: float64


While having nearly 40% of our features off-limits immediately seems frustrating, we can do something about a few of these. Notice quite a few of our features have just 5 NaNs--probably attributed to missing data on a few certain players. Closer inspection should confirm this:

In [19]:
per_100[per_100.PER.isnull()]

,Last Name,First Name,Full Name,Year,Pos,Age,Tm,G,GS,MP,...,TOV%,USG%,OWS,DWS,WS,WS/48,OBPM,DBPM,BPM,VORP
4059,James,Damion,Damion James,2012.0,SF,25.0,BRK,2.0,0.0,0.0,...,NaN,NaN,0.0,0.0,0.0,NaN,-5.5,0.3,-5.2,0.0
5675,Curry,JamesOn,JamesOn Curry,2009.0,PG,24.0,LAC,1.0,0.0,0.0,...,NaN,NaN,0.0,0.0,0.0,NaN,-6.7,-0.2,-6.9,0.0
7043,Lasme,Stéphane,Stéphane Lasme,2007.0,SF,25.0,GSW,1.0,0.0,0.0,...,NaN,NaN,0.0,0.0,0.0,NaN,-6.0,-0.4,-6.4,0.0
7951,Davis,Josh,Josh Davis,2005.0,PF,25.0,HOU,1.0,0.0,0.0,...,NaN,NaN,0.0,0.0,0.0,NaN,-7.2,0.9,-6.3,0.0
8268,Scales,Alex,Alex Scales,2005.0,SG,27.0,SAS,1.0,0.0,0.0,...,NaN,NaN,0.0,0.0,0.0,NaN,-6.0,1.6,-4.4,0.0


Sure enough, these 5 players are the problem. Considering their stats are more or less inconsequential to our end goal, we can simply drop them.

In [20]:
nanplayers = per_100[per_100.PER.isnull()].index

In [21]:
per_100 = per_100.drop(nanplayers)
nanlist = per_100.isnull().sum()
nanlist = nanlist.where(nanlist > 0).dropna()
print('Number of features containing NaN values:',(len(nanlist)),'\n')
print(nanlist)

Number of features containing NaN values: 10 

GS       694.0
FG%       93.0
3P%     3618.0
2P%      145.0
FT%      841.0
ORtg      50.0
TS%       81.0
3PAr      93.0
FTr       93.0
TOV%      66.0
dtype: float64


Thankfully, shooting percentages are going to be consistent between this two sets of data, so we don't need to worry about the NaNs we see above contributing too too much to our earlier list; same with the 694 NaNs under 'GS'--that information is now stored in players.csv.

However, there is an important feature that needs our attention still, one specific to this data set, and that is "ORtg", or Offensive Rating. This composite statistic will be explored later, but for now, understand it be simply: per 100 possessions, how many points is a player likely to generate when he tries?"

In [22]:
per_100[per_100.ORtg.isnull()].loc[:,'MP'].describe()

count    50.000000
mean      2.780000
std       2.297203
min       1.000000
25%       1.000000
50%       2.000000
75%       3.750000
max      13.000000
Name: MP, dtype: float64

The above code gives us some descriptive statistics for the crucial feature these NaN records share: MP or minutes played all season for each of them is extremely low. The player with the most minutes in this list played for merely 13 minutes (every game, of which there are 82, is 48 minutes long). We can drop these players, as it would be very unlikely that their stats contribute to our exploratory analysis of the data.

In [23]:
dontplay = per_100[per_100.ORtg.isnull()].index

per_100 = per_100.drop(dontplay)

nanlist = per_100.isnull().sum()
nanlist = nanlist.where(nanlist > 0).dropna()
print('Number of features containing NaN values:',(len(nanlist)),'\n')
print(nanlist)

Number of features containing NaN values: 9 

GS       694.0
FG%       43.0
3P%     3568.0
2P%       95.0
FT%      791.0
TS%       31.0
3PAr      43.0
FTr       43.0
TOV%      16.0
dtype: float64


Because all of these features remaining have been accounted for with the first data set, we left with essentially zero unaccounted for NaN values in this one. We now save this cleaned up version for EDA.

Added late into the data-procuring process was salary information which can be found in the notebook 'salary-data.ipynb'. We'll want this information merged with our two player-level dataframes, players and per_100.

In [33]:
salaries = pd.read_csv('temp_csvs/salaries.csv')
salaries = salaries.drop('Unnamed: 0', axis = 1)
salaries = salaries.rename(columns = {'name':'Full Name',
                                      'year':'Year'})
salaries.head()

,Full Name,height,weight,shoots,salary,Year
0,Alaa Abdelnaby,82.0,240.0,Right,395000,1990
1,Alaa Abdelnaby,82.0,240.0,Right,494000,1991
2,Alaa Abdelnaby,82.0,240.0,Right,500000,1992
3,Alaa Abdelnaby,82.0,240.0,Right,805000,1993
4,Alaa Abdelnaby,82.0,240.0,Right,650000,1994


In [25]:
pergame_salaries = pd.merge(pergame, salaries, on = ['Full Name','Year'], how = 'outer')
pergame_salaries.describe()

,Year,Age,G,GS,MPG,FG,FGA,FG%,3P,3PA,...,TRB,AST,STL,BLK,TOV,PF,PTS,height,weight,salary
count,23121.000000,21091.000000,21091.000000,21091.000000,21091.000000,21091.000000,21091.000000,20979.000000,21091.000000,21091.000000,...,21091.000000,21091.000000,21091.000000,21091.000000,21091.000000,21091.000000,21091.000000,17228.000000,17228.000000,1.796200e+04
mean,2001.000606,26.836376,48.336636,23.412048,19.645593,3.012356,6.670077,0.437063,0.406894,1.191067,...,3.413214,1.825897,0.648936,0.396819,1.224627,1.891404,7.930544,78.983051,217.362027,3.213576e+06
std,11.163348,4.023063,26.974387,28.590484,10.139366,2.220074,4.596676,0.100506,0.589091,1.564000,...,2.515093,1.822386,0.474748,0.499504,0.816348,0.879406,5.908232,3.722955,27.592768,4.354517e+06
min,1979.000000,18.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,63.000000,133.000000,2.706000e+03
25%,1992.000000,24.000000,24.000000,0.000000,11.200000,1.300000,3.100000,0.400000,0.000000,0.000000,...,1.600000,0.600000,0.300000,0.100000,0.600000,1.200000,3.300000,76.000000,195.000000,5.908500e+05
50%,2002.000000,26.000000,53.000000,8.000000,18.800000,2.400000,5.500000,0.443000,0.100000,0.400000,...,2.800000,1.200000,0.600000,0.200000,1.000000,1.900000,6.400000,79.000000,218.000000,1.498500e+06
75%,2011.000000,30.000000,74.000000,45.000000,27.900000,4.300000,9.400000,0.486000,0.600000,1.900000,...,4.600000,2.500000,0.900000,0.500000,1.700000,2.500000,11.300000,82.000000,235.000000,3.940240e+06
max,2018.000000,44.000000,85.000000,83.000000,44.500000,13.400000,27.800000,1.000000,5.100000,13.200000,...,18.700000,14.500000,3.700000,6.000000,5.700000,6.000000,37.100000,91.000000,360.000000,3.745715e+07


In [26]:
per_100_salaries = pd.merge(per_100, salaries, on=['Full Name', 'Year'], how = 'outer')
per_100_salaries.describe()

,Year,Age,G,GS,MP,FG,FGA,FG%,3P,3PA,...,DWS,WS,WS/48,OBPM,DBPM,BPM,VORP,height,weight,salary
count,23076.000000,21028.000000,21028.000000,20334.000000,21028.000000,21028.000000,21028.000000,20979.000000,21028.000000,21028.000000,...,21028.000000,21028.000000,21028.000000,21028.000000,21028.000000,21028.000000,21028.000000,17201.000000,17201.000000,1.793400e+04
mean,2000.995580,26.839405,48.477744,22.722927,1126.481596,7.171643,16.462479,0.437063,0.977344,2.990308,...,1.137636,2.316255,0.064531,-1.829251,-0.578571,-2.407466,0.525433,78.980931,217.350677,3.216959e+06
std,11.166987,4.024103,26.891091,28.254984,915.893537,2.721593,5.171864,0.100506,1.335646,3.508062,...,1.194383,2.897222,0.106274,3.938655,2.297184,4.831366,1.303307,3.724009,27.596419,4.356115e+06
min,1979.000000,18.000000,1.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,-1.000000,-2.100000,-2.519000,-73.800000,-30.400000,-86.700000,-2.600000,63.000000,133.000000,2.706000e+03
25%,1992.000000,24.000000,24.000000,0.000000,285.750000,5.500000,13.100000,0.400000,0.000000,0.100000,...,0.200000,0.200000,0.031000,-3.500000,-1.700000,-4.200000,-0.200000,76.000000,195.000000,5.948770e+05
50%,2002.000000,26.000000,53.000000,7.000000,937.000000,7.100000,16.200000,0.443000,0.300000,1.400000,...,0.800000,1.200000,0.075000,-1.500000,-0.500000,-1.900000,0.000000,79.000000,218.000000,1.499187e+06
75%,2011.000000,30.000000,74.000000,42.000000,1837.000000,8.700000,19.600000,0.486000,1.800000,5.400000,...,1.700000,3.500000,0.114000,0.300000,0.700000,0.200000,0.800000,82.000000,235.000000,3.941750e+06
max,2018.000000,44.000000,85.000000,83.000000,3533.000000,52.600000,65.500000,1.000000,50.100000,52.900000,...,9.100000,21.200000,2.712000,68.600000,30.500000,54.400000,12.400000,91.000000,360.000000,3.745715e+07


In [27]:
per_100_salaries.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 23076 entries, 0 to 23075
Data columns (total 57 columns):
Last Name     21028 non-null object
First Name    21028 non-null object
Full Name     23076 non-null object
Year          23076 non-null float64
Pos           21028 non-null object
Age           21028 non-null float64
Tm            21028 non-null object
G             21028 non-null float64
GS            20334 non-null float64
MP            21028 non-null float64
FG            21028 non-null float64
FGA           21028 non-null float64
FG%           20979 non-null float64
3P            21028 non-null float64
3PA           21028 non-null float64
3P%           17317 non-null float64
2P            21028 non-null float64
2PA           21028 non-null float64
2P%           20913 non-null float64
FT            21028 non-null float64
FTA           21028 non-null float64
FT%           20140 non-null float64
ORB           21028 non-null float64
DRB           21028 non-null float64
TRB     

In [28]:
pergame_salaries.to_csv('data/pergame_clean.csv')

per_100_salaries.to_csv('data/per_100_clean.csv')

In [ ]:
#######################################################################################################################

In [30]:
shooting = pd.read_csv('temp_csvs/shooting.csv', sep = '\t') # one of the few tab-delimited csv files
shooting = shooting.drop(columns = 'Unnamed: 0')
shooting.head()

,Full Name,Year,Tm,FG%,Dist.,2P_A,0-3_A,3-10_A,10-16_A,16-3pt_A,...,16-3pt%,3P%,Astd_2P,Dunk%,Dunks_Md,Astd_3P,%Corner3_A,Corner_3P%,Heaves_A,Heaves_Md
0,Mahmoud Abdul-Rauf,2000,VAN,0.488,17.7,0.943,0.028,0.049,0.171,0.695,...,0.497,0.286,0.543,0.000,0.0,0.5,0.071,0.0,0.0,0.0
1,Tariq Abdul-Wahad,2000,DEN,0.387,11.4,0.910,0.288,0.171,0.135,0.315,...,0.371,0.400,0.641,0.027,3.0,1.0,0.200,1.0,0.0,0.0
2,Tariq Abdul-Wahad,2001,TOT,0.374,8.1,0.986,0.395,0.218,0.129,0.245,...,0.111,0.500,0.667,0.068,8.0,1.0,0.000,NaN,0.0,0.0
3,Tariq Abdul-Wahad,2001,DEN,0.379,8.0,0.986,0.400,0.221,0.117,0.248,...,0.111,0.500,0.667,0.069,8.0,1.0,0.000,NaN,0.0,0.0
4,Tariq Abdul-Wahad,2001,DAL,0.000,13.5,1.000,0.000,0.000,1.000,0.000,...,NaN,NaN,NaN,0.000,0.0,NaN,NaN,NaN,0.0,0.0


In [31]:
final = pd.merge(per_100_salaries, shooting, on = ['Full Name','Year','Tm'], how = 'left')

final = final.rename(columns = {'FG%_x' :'FG%',
                                '3P%_x' :'3P%',
                                '2P%_x' :'2P%',
                                'FG%_y' :'drop',
                                '3P%_y' :'drop2',
                                '2P%_y' :'drop3'})

final = final.drop(columns = ['drop','drop2','drop3'])

In [32]:
file_loc = 'data/finaldata.csv'

final.to_csv(file_loc)